In [5]:
import numpy as np
import pandas as pd
import requests as rq
import time, datetime
import json

#https://docs.cloud.coinbase.com/exchange/docs/welcome

In [2]:
#Fetch data length
url = 'https://api.exchange.coinbase.com/products/{}/candles?granularity=86400'

#Fetch dataset with given time interval
url2 = 'https://api.exchange.coinbase.com/products/{}/candles?granularity=86400&start={}&end={}'

In [4]:
ticker = 'BTC-USD'

In [13]:
#Calculates time length in each batch
def delta(u):
    resp = rq.get(u.format(ticker)).json()
    dt = resp[0][0] = resp[-1][0]
    return dt

In [14]:
#Convert numeric timestamp to ISO time format to be sent to Coinbase
def timestamp(x, conv='%Y-%m-%dT%H:%M:%SZ'):
    return datetime.datetime.fromtimestamp(x).strftime(conv)

In [16]:
#Batch seconds
dT = delta(url)

#Initial timestamps
t1 = int(time.time())
t0 = t1 - dT

#Number of batches we wish to fetch
years = 6

#Table too hold data
table = []

In [21]:
#For loop too get all info
for t in range(years):
    #Show how much is left to pull
    print(years - t, 'pulling')
    
    #Set up ISO timestamps
    t0, t1 = timestamp(t0), timestamp(t1)
    
    #Building table
    table = table + rq.get(url2.format(ticker, t0, t1)).json()
    
    #Sleep timer too not get kicked
    time.sleep(1)
    
    #Subtracting timestamps
    t0 = t0 - dT
    t1 = t1 - dT

6 pulling


TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
dataframe = pd.DataFrame(table, columns = ['Time','Low','High','Open','Close','Volume'])

print(dataframe)